<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/PJ2_ML-DL%20for%205%20class%20WSN-DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid RF-LSTM for 5 class classifications**

In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
# Load data
data = pd.read_csv('Dataset_P2.csv')
data.isnull().sum()

 id                 0
 Time               0
 Is_CH              0
 who CH             0
 Dist_To_CH         0
 ADV_S              0
 ADV_R              0
 JOIN_S             0
 JOIN_R             0
 SCH_S              0
 SCH_R              0
Rank                0
 DATA_S             0
 DATA_R             0
 Data_Sent_To_BS    0
 dist_CH_To_BS      0
 send_code          0
Expaned Energy      0
Class               0
dtype: int64

In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374661 entries, 0 to 374660
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0    id               374661 non-null  int64  
 1    Time             374661 non-null  int64  
 2    Is_CH            374661 non-null  int64  
 3    who CH           374661 non-null  int64  
 4    Dist_To_CH       374661 non-null  float64
 5    ADV_S            374661 non-null  int64  
 6    ADV_R            374661 non-null  int64  
 7    JOIN_S           374661 non-null  int64  
 8    JOIN_R           374661 non-null  int64  
 9    SCH_S            374661 non-null  int64  
 10   SCH_R            374661 non-null  int64  
 11  Rank              374661 non-null  int64  
 12   DATA_S           374661 non-null  int64  
 13   DATA_R           374661 non-null  int64  
 14   Data_Sent_To_BS  374661 non-null  int64  
 15   dist_CH_To_BS    374661 non-null  float64
 16   send_code        37

In [17]:
# Preprocessing
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
le = LabelEncoder()
y = le.fit_transform(y)
sc = StandardScaler()
X = sc.fit_transform(X)
X = np.reshape(X, (X.shape, X.shape[1], 1))
y = to_categorical(y)

TypeError: ignored

In [19]:
from keras.utils import np_utils
encoder = LabelEncoder()
y = encoder.fit_transform(dataset['Class'])
y = np_utils.to_categorical(y)
X = dataset.drop(['Class'],axis=1)
X=X.values
print(X.shape)
print(y.shape)

(374661, 18)
(374661, 5)


In [20]:
# Data preprocessing
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [21]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Build Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train.reshape(X_train.shape, -1), np.argmax(y_train, axis=1))

# Hybrid model prediction
y_pred_lstm = model.predict(X_test)
y_pred_rf = rf.predict(X_test.reshape(X_test.shape, -1))
y_pred = np.argmax((y_pred_lstm + y_pred_rf)/2, axis=1)

# Evaluation metrics
print('Accuracy:', accuracy_score(np.argmax(y_test, axis=1), y_pred))
print('Confusion Matrix:\n', confusion_matrix(np.argmax(y_test, axis=1), y_pred))
print('Classification Report:\n', classification_report(np.argmax(y_test, axis=1), y_pred))

Epoch 1/10
42509/42509 [==============================] - 919s 22ms/step - loss: 0.1845 - accuracy: 0.9093
Epoch 2/10
42509/42509 [==============================] - 910s 21ms/step - loss: 0.0717 - accuracy: 0.9768
Epoch 3/10
42509/42509 [==============================] - 903s 21ms/step - loss: 0.0616 - accuracy: 0.9796
Epoch 4/10
42509/42509 [==============================] - 905s 21ms/step - loss: 0.0568 - accuracy: 0.9811
Epoch 5/10
42509/42509 [==============================] - 905s 21ms/step - loss: 0.0530 - accuracy: 0.9822
Epoch 6/10
42509/42509 [==============================] - 909s 21ms/step - loss: 0.0503 - accuracy: 0.9831
Epoch 7/10
42509/42509 [==============================] - 917s 22ms/step - loss: 0.0490 - accuracy: 0.9834
Epoch 8/10
42509/42509 [==============================] - 907s 21ms/step - loss: 0.0481 - accuracy: 0.9838
Epoch 9/10
42509/42509 [==============================] - 912s 21ms/step - loss: 0.0471 - accuracy: 0.9841
Epoch 10/10
42509/42509 [============

TypeError: ignored

In [23]:
# Build Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
##rf.fit(X_train.reshape(X_train.shape, -1), np.argmax(y_train, axis=1))
rf.fit(X_train.reshape(X_train.shape[0], -1), np.argmax(y_train, axis=1))


# Hybrid model prediction
y_pred_lstm = model.predict(X_test)
y_pred_rf = rf.predict(X_test.reshape(X_test.shape, -1))
y_pred = np.argmax((y_pred_lstm + y_pred_rf)/2, axis=1)

# Evaluation metrics
print('Accuracy:', accuracy_score(np.argmax(y_test, axis=1), y_pred))
print('Confusion Matrix:\n', confusion_matrix(np.argmax(y_test, axis=1), y_pred))
print('Classification Report:\n', classification_report(np.argmax(y_test, axis=1), y_pred))

10628/10628 [==============================] - 77s 7ms/step


TypeError: ignored